# 自定义层  
有时
我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。  

### 1. 不带参数的层  
要构建它，我们只需继承基础层类并实现前向传播功能。  

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLyaer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLyaer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

# 可以将该层放入更复杂的块中  
net = nn.Sequential(nn.Linear(8, 10), CenteredLyaer())

tensor([-2., -1.,  0.,  1.,  2.])

### 2. 带参数的层  
可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。比如管理访问、初始化、共享、保存和加载模型参数。  

**层与块的区别：块中只定义层，而不显示地定义层的参数；层，需要显示地定义参数**。

现在，让我们实现自定义版本的全连接层。回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。在此实现中，我们使用修正线性单元作为激活函数。该层需要输入参数： in_units和units，分别表示输入数和输出数。

In [9]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 手动进行参数初始化
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.6690, -0.5229,  1.2833],
        [-0.0706, -1.4746,  0.2132],
        [-0.6567,  1.4381,  0.2273],
        [-2.3541,  0.2999, -0.5144],
        [ 0.0479,  1.1698, -0.6599]], requires_grad=True)

我们可以使用自定义层直接执行前向传播计算。  

In [10]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.6251, 0.4752],
        [0.0000, 0.0000, 0.0214]])

我们还可以使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层。  

In [11]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.1652],
        [0.0000]])